# Wykrywanie społeczności w sieciach

**Dominika Szydło** i **Oliwier Kaszyca**

---

In [9]:
import pandas as pd
from pathlib import Path
from gensim.models import Word2Vec

In [8]:
DATA_PATH = Path("data")
EMBBEDDINGS_PATH = Path("embbeddings")

---

# Opis problemu

## Typ problemu

Rozważany problem to klasyfikacja wierzchołków w grafie reprezentującym sieć społecznościową. 


## Zbiór danych


Badania przeprowadzono na zmodyfikowanej sieci *email-Eu-core* składającym się z 986 wierzchołków, przyporządkowanych do 42 grup. Wizualizację badanej sieci przedstawiono poniżej.

![Network](images/network.png)

### Dane wejściowe

Danymi wejściowymi będzie informacja o pracowniku, reprezentowana jako wektor wygenerowany przez metodę [`node2vec`](https://github.com/aditya-grover/node2vec) ($X$) oraz informacja o przynależności do działu ($Y$).


In [7]:
pd.read_csv(DATA_PATH / "train_data.tsv", sep=" ")

,user_id,department_id,embbedding_id
0,0,1,266
1,1,1,215
2,2,21,60
3,3,21,128
4,4,21,63
...,...,...,...
981,1000,4,759
982,1001,21,674
983,1002,1,924
984,1003,6,974


In [12]:
model = Word2Vec.load(str(EMBBEDDINGS_PATH / "graph.model"))
model.wv[0]

array([ 0.02225889,  0.19447829, -0.09200738, -0.27720863,  0.03538975,
        0.22662188,  0.18581073, -0.0903049 ,  0.12872799,  0.00794893,
        0.28711984, -0.17070821, -0.2333458 ,  0.24912564, -0.2353019 ,
        0.03209164,  0.12682602, -0.04706044, -0.0633488 , -0.28577852,
       -0.13404648,  0.21503699, -0.00330228,  0.01003238,  0.30007756,
       -0.00545652,  0.23217416,  0.2720985 , -0.33044073,  0.2142734 ,
       -0.10560817, -0.36481318,  0.12313964, -0.10946704,  0.19214727,
        0.12820779,  0.02071633, -0.10759116,  0.03678202,  0.08680518,
        0.33808035, -0.03600693, -0.02475484, -0.09358548, -0.20100296,
        0.18724719,  0.26912943, -0.06421999,  0.06172297,  0.06551058,
       -0.13186899, -0.09475884, -0.05098552,  0.17380567, -0.03078185,
       -0.17430164, -0.15307337,  0.0892554 , -0.33101523,  0.08577861,
       -0.08642791,  0.2687027 , -0.12943153, -0.16547027], dtype=float32)

### Dane wejściowe

Jako dane wyjściowe traktujemy parę $(n, c)$, gdzie $n$ to ID pracownika (wierzchołka), a $c$ to jego przynależność do działu (klasy).

## Hipoteza badawcza

TODO

# Opis wybranych modeli

## Graph Attention Network (GAT)

TODO

## Graph Convolutional Network (GCN) 

TODO

# Eksperymenty

TODO

# Opis rezultatów

TODO